# Phase-1

In [12]:
# CELL 0A: Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
# CELL 0B: Dataset ZIP path

zip_path = "/content/drive/MyDrive/sleep-edf-database-1.0.0.zip"

In [14]:
# CELL 0C: Unzip dataset

import zipfile
import os

extract_path = "/content/sleep_edf_data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted to:", extract_path)

Dataset extracted to: /content/sleep_edf_data


In [15]:
# CELL 0D: List extracted files

for root, dirs, files in os.walk(extract_path):
    for file in files[:10]:  # show only first 10
        print(os.path.join(root, file))

/content/sleep_edf_data/sleep-edf-database-1.0.0/sc4112e0.hyp
/content/sleep_edf_data/sleep-edf-database-1.0.0/st7022j0.hyp
/content/sleep_edf_data/sleep-edf-database-1.0.0/st7022j0.rec
/content/sleep_edf_data/sleep-edf-database-1.0.0/st7121j0.rec
/content/sleep_edf_data/sleep-edf-database-1.0.0/st7121j0.hyp
/content/sleep_edf_data/sleep-edf-database-1.0.0/st7132j0.rec
/content/sleep_edf_data/sleep-edf-database-1.0.0/sc4102e0.rec
/content/sleep_edf_data/sleep-edf-database-1.0.0/RECORDS
/content/sleep_edf_data/sleep-edf-database-1.0.0/sc4102e0.hyp
/content/sleep_edf_data/sleep-edf-database-1.0.0/st7132j0.hyp


In [16]:
# CELL 0: Install required packages
# Run this only ONCE

# CELL 0: Install required packages (run once)

!pip install mne numpy scipy scikit-learn pyedflib

In [17]:
# CELL 1: Import libraries

import numpy as np
import mne
from scipy.signal import butter, filtfilt
from scipy.fft import fft

In [18]:
# CELL 2: Load EEG data

def load_eeg(psg_file):
    """
    Loads EEG signals from a PSG EDF file.
    Returns EEG data and sampling frequency.
    """
    raw = mne.io.read_raw_edf(psg_file, preload=True, verbose=False)

    # Use only EEG channels (ignore EOG, EMG)
    eeg_channels = ['Fpz-Cz', 'Pz-Oz']
    raw.pick_channels(eeg_channels)

    fs = raw.info['sfreq']      # Sampling frequency (100 Hz)
    data = raw.get_data()       # Shape: (channels, samples)

    return data, fs

In [19]:
# CELL 3: Load hypnogram labels

def load_hypnogram(hyp_file):
    """
    Loads sleep stage annotations from Hypnogram EDF+ file.
    """
    annotations = mne.read_annotations(hyp_file)

    labels = []
    for desc in annotations.description:
        labels.append(desc)

    return labels

In [20]:
# CELL 4: Band-pass filter

def bandpass_filter(signal, fs, low=0.5, high=30):
    """
    Filters EEG signal between 0.5–30 Hz.
    """
    nyq = 0.5 * fs
    b, a = butter(4, [low/nyq, high/nyq], btype='band')
    return filtfilt(b, a, signal)

In [21]:
# CELL 5: Windowing EEG

def split_into_windows(signal, fs, window_sec=1):
    """
    Splits EEG signal into 1-second windows.
    """
    window_size = int(fs * window_sec)
    windows = []

    for start in range(0, len(signal) - window_size, window_size):
        windows.append(signal[start:start + window_size])

    return windows

In [22]:
# CELL 6: FFT-based band power extraction

def extract_band_power(window, fs):
    """
    Extracts Delta, Theta, Alpha, Beta power from EEG window.
    """
    fft_vals = np.abs(fft(window))**2
    freqs = np.fft.fftfreq(len(fft_vals), 1/fs)

    def band(low, high):
        idx = np.where((freqs >= low) & (freqs <= high))
        return np.mean(fft_vals[idx])

    delta = band(0.5, 4)
    theta = band(4, 8)
    alpha = band(8, 13)
    beta  = band(13, 30)

    return [delta, theta, alpha, beta]

In [23]:
# CELL 7: Label mapping

def map_label(stage):
    """
    Maps Sleep-EDF stages to project classes.
    """
    if stage == 'W':
        return 'Relaxed'
    elif stage in ['1', 'R']:
        return 'Drowsy'
    elif stage in ['2', '3', '4']:
        return 'Sleep'
    else:
        return None   # Ignore M, ?

In [26]:
# CELL 8: Dataset creation from folder

import glob

def create_full_dataset(data_folder):
    X, y = [], []

    psg_files = sorted(glob.glob(os.path.join(data_folder, "*PSG.edf")))

    for psg_file in psg_files:
        hyp_file = psg_file.replace("PSG.edf", "Hypnogram.edf")
        if not os.path.exists(hyp_file):
            continue

        features, labels = create_dataset(psg_file, hyp_file)
        X.extend(features)
        y.extend(labels)

    return np.array(X), np.array(y)

In [27]:
# CELL 9: Run dataset creation

X, y = create_full_dataset(extract_path)

print("Features shape:", X.shape)
print("Labels count:", len(y))

Features shape: (0,)
Labels count: 0


# Phase-2

In [31]:
# STEP 1: Check current working directory
import os

print("Current directory:", os.getcwd())
print("Files here:", os.listdir())


Current directory: /content
Files here: ['.config', 'sleep_edf_data', 'drive', 'sample_data']


In [32]:
# STEP 2: Search entire workspace for .rec files

import os

found = []

for root, dirs, files in os.walk("/"):
    for file in files:
        if file.endswith(".rec"):
            found.append(os.path.join(root, file))

print("Total .rec files found:", len(found))
for f in found[:5]:
    print(f)


Total .rec files found: 8
/content/sleep_edf_data/sleep-edf-database-1.0.0/st7022j0.rec
/content/sleep_edf_data/sleep-edf-database-1.0.0/st7121j0.rec
/content/sleep_edf_data/sleep-edf-database-1.0.0/st7132j0.rec
/content/sleep_edf_data/sleep-edf-database-1.0.0/sc4102e0.rec
/content/sleep_edf_data/sleep-edf-database-1.0.0/sc4002e0.rec


In [34]:
# Cell-3A: Rename .rec file to .edf (safe copy)

import shutil
import os

rec_file = "/content/sleep_edf_data/sleep-edf-database-1.0.0/sc4002e0.rec"
edf_file = rec_file.replace(".rec", ".edf")

# Copy only if not already created
if not os.path.exists(edf_file):
    shutil.copy(rec_file, edf_file)

print("EDF file created:", edf_file)

EDF file created: /content/sleep_edf_data/sleep-edf-database-1.0.0/sc4002e0.edf


In [35]:
# Cell-3B: Load EDF file correctly using MNE

import mne

raw = mne.io.read_raw_edf(edf_file, preload=True, verbose=False)

print("Channels:", raw.ch_names)
print("Sampling frequency:", raw.info['sfreq'])
print("Total duration (seconds):", raw.times[-1])

Channels: ['EEG Fpz-Cz', 'EEG Pz-Oz', 'EOG horizontal', 'Resp oro-nasal', 'EMG Submental', 'Temp body', 'Event marker']
Sampling frequency: 100.0
Total duration (seconds): 84899.99


# Phase-3

In [37]:
# Cell-4A: Rename .hyp file to .edf so MNE can read it

import shutil
import os

hyp_file = "/content/sleep_edf_data/sleep-edf-database-1.0.0/sc4002e0.hyp"
hyp_edf = hyp_file.replace(".hyp", ".edf")

if not os.path.exists(hyp_edf):
    shutil.copy(hyp_file, hyp_edf)

print("Hypnogram EDF created:", hyp_edf)

Hypnogram EDF created: /content/sleep_edf_data/sleep-edf-database-1.0.0/sc4002e0.edf


In [38]:
# Cell-4B: Load sleep stage annotations

import mne

annotations = mne.read_annotations(hyp_edf)

print("Number of annotations:", len(annotations))
print("First 10 annotations:")
print(annotations[:10])

Number of annotations: 0
First 10 annotations:
<Annotations | 0 segments>


In [39]:
# Cell-4C: Read hypnogram using pyEDFlib (correct method)

import pyedflib
import numpy as np

hyp_reader = pyedflib.EdfReader(hyp_file)

print("Signals in hypnogram:", hyp_reader.getSignalLabels())
print("Number of signals:", hyp_reader.signals_in_file)
print("Duration (seconds):", hyp_reader.getFileDuration())

Signals in hypnogram: ['Hypnogram']
Number of signals: 1
Duration (seconds): 84930.0


In [40]:
# Cell-4D: Extract sleep stage signal

sleep_stage_signal = hyp_reader.readSignal(0)

print("Sleep stage vector shape:", sleep_stage_signal.shape)
print("Unique stage codes:", np.unique(sleep_stage_signal))

Sleep stage vector shape: (2831,)
Unique stage codes: [0. 1. 2. 3. 4. 5. 6. 9.]


# Phase-4

In [41]:
# Cell-5A: Pick EEG channels only

eeg_channels = [ch for ch in raw.ch_names if 'EEG' in ch]

raw_eeg = raw.copy().pick_channels(eeg_channels)

print("Selected EEG channels:", raw_eeg.ch_names)

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Selected EEG channels: ['EEG Fpz-Cz', 'EEG Pz-Oz']


In [42]:
# Cell-5B: Create fixed-length 30-second epochs

epochs = mne.make_fixed_length_epochs(
    raw_eeg,
    duration=30.0,
    preload=True,
    overlap=0
)

print("Number of EEG epochs:", len(epochs))
print("Epoch shape:", epochs.get_data().shape)

Not setting metadata
2830 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 2830 events and 3000 original time points ...
0 bad epochs dropped
Number of EEG epochs: 2830
Epoch shape: (2830, 2, 3000)


In [43]:
# Cell-5C: Align EEG epochs with sleep stage labels

import numpy as np

min_len = min(len(epochs), len(sleep_stage_signal))

epochs_data = epochs.get_data()[:min_len]
labels = sleep_stage_signal[:min_len]

print("Aligned EEG shape:", epochs_data.shape)
print("Aligned labels shape:", labels.shape)

Aligned EEG shape: (2830, 2, 3000)
Aligned labels shape: (2830,)


In [44]:
# Cell-5D: Remove invalid sleep stages

valid_idx = np.where((labels != 6) & (labels != 9))[0]

epochs_data = epochs_data[valid_idx]
labels = labels[valid_idx]

print("Final EEG epochs shape:", epochs_data.shape)
print("Final labels count:", len(labels))
print("Remaining sleep stages:", np.unique(labels))

Final EEG epochs shape: (2829, 2, 3000)
Final labels count: 2829
Remaining sleep stages: [0. 1. 2. 3. 4. 5.]


# Phase-5

In [62]:
# Cell-6A: Define EEG frequency bands

bands = {
    "delta": (0.5, 4),
    "theta": (4, 8),
    "alpha": (8, 13),
    "beta": (13, 30)
}

print("Defined frequency bands:", bands)

Defined frequency bands: {'delta': (0.5, 4), 'theta': (4, 8), 'alpha': (8, 13), 'beta': (13, 30)}


In [63]:
# Cell-6B-Updated: Extract band power features per EEG channel

import numpy as np
from scipy.signal import welch

sfreq = raw.info['sfreq']  # Sampling frequency (100 Hz)

def extract_band_power_channels(epoch_signal, sfreq):
    """
    epoch_signal: shape (channels, samples)
    returns: [Δ1, Θ1, Α1, Β1, Δ2, Θ2, Α2, Β2]
    """
    features = []
    for ch_signal in epoch_signal:  # Loop through EEG channels
        freqs, psd = welch(ch_signal, sfreq, nperseg=1024)
        for band in bands.values():
            idx = np.logical_and(freqs >= band[0], freqs <= band[1])
            band_power = np.sum(psd[idx])
            features.append(band_power)
    return features

In [64]:
# Cell-6C-Updated: Extract channel-wise features for all epochs

features = []
for epoch in epochs_data:
    feat = extract_band_power_channels(epoch, sfreq)
    features.append(feat)

X = np.array(features)
y = labels

print("Feature matrix shape (channel-wise):", X.shape)
print("Label vector shape:", y.shape)

Feature matrix shape (channel-wise): (2829, 8)
Label vector shape: (2829,)


In [65]:
# Cell-6D: Map sleep stages to project outputs

def map_stage(stage):
    if stage == 0:
        return 1   # Stress / High Focus
    elif stage in [1, 2]:
        return 2   # Drowsy
    elif stage in [3, 4]:
        return 3   # Sleep
    elif stage == 5:
        return 0   # Relaxed
    else:
        return -1

y_mapped = np.array([map_stage(s) for s in y])

print("Mapped class labels:", np.unique(y_mapped))

Mapped class labels: [0 1 2 3]


# Phase-6

In [66]:
# Cell-7A: Normalize features

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X)

print("Feature range after normalization:")
print("Min:", X_norm.min(), "Max:", X_norm.max())

Feature range after normalization:
Min: 0.0 Max: 1.0


In [67]:
# Cell-7B: One-hot encode output labels

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)  # use sparse_output instead of sparse
y_onehot = encoder.fit_transform(y_mapped.reshape(-1, 1))

print("One-hot label shape:", y_onehot.shape)

One-hot label shape: (2829, 4)


In [68]:
# Cell-7C: Split dataset

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_norm, y_onehot, test_size=0.2, random_state=42, stratify=y_mapped
)

print("Training samples:", X_train.shape[0])
print("Testing samples:", X_test.shape[0])

Training samples: 2263
Testing samples: 566


In [69]:
# Cell-7D-Updated: Define ANN for 8-channel features

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(16, activation='sigmoid', input_shape=(8,)),  # Increased hidden neurons
    Dense(4, activation='sigmoid')  # 4 output classes (Relaxed, Stress, Drowsy, Sleep)
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │            68 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 212 (848.00 B)

 Trainable params: 212 (848.00 B)

 Non-trainable params: 0 (0.00 B)

In [70]:
# Cell-7E-Weighted: Train with class weights

from sklearn.utils import class_weight

# Compute weights
weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_mapped),
    y=y_mapped
)
class_weights = dict(enumerate(weights))

history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    class_weight=class_weights,  # Apply class balancing
    verbose=1
)

Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - accuracy: 0.0766 - loss: 1.3587 - val_accuracy: 0.1145 - val_loss: 1.3887
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2117 - loss: 1.3659 - val_accuracy: 0.7181 - val_loss: 1.3437
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7086 - loss: 1.3800 - val_accuracy: 0.7181 - val_loss: 1.3133
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7893 - loss: 1.3288 - val_accuracy: 0.7621 - val_loss: 1.2862
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7980 - loss: 1.3158 - val_accuracy: 0.7489 - val_loss: 1.2702
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8041 - loss: 1.3104 - val_accuracy: 0.7753 - val_loss: 1.2287
Epoch 7/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8274 - loss: 1.2800 - val_accuracy: 0.7621 - val_loss: 1.2062
Epoch 8/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8166 - loss: 1.2176 - val_accuracy: 0.7577 - val_loss

# Phase-7

In [71]:
# Cell-8A: Extract trained weights and biases

weights = []
biases = []

for layer in model.layers:
    w, b = layer.get_weights()
    weights.append(w)
    biases.append(b)

# Show shapes
for i, (w, b) in enumerate(zip(weights, biases)):
    print(f"Layer {i}: Weights {w.shape}, Biases {b.shape}")

Layer 0: Weights (8, 16), Biases (16,)
Layer 1: Weights (16, 4), Biases (4,)


In [72]:
# Cell-8B: Convert weights/biases to fixed-point integers

def to_fixed_point(array, bits=16, frac_bits=8):
    """Convert floating-point array to fixed-point integers"""
    scale = 2 ** frac_bits
    fixed = np.round(array * scale).astype(int)
    return fixed

weights_fixed = [to_fixed_point(w) for w in weights]
biases_fixed = [to_fixed_point(b) for b in biases]

In [78]:
# Cell-8C-Fixed: Verilog-friendly arrays

def verilog_array(name, array):
    """
    Converts a numpy array to a Verilog reg signed [15:0] array
    """
    flat = array.flatten()
    s = f"// {name}\n"
    s += f"reg signed [15:0] {name} [0:{len(flat)-1}] = '{{"
    s += ', '.join(map(str, flat))
    s += "}};\n"
    return s

for i, (w, b) in enumerate(zip(weights_fixed, biases_fixed)):
    print(verilog_array(f"layer{i}_weights", w))
    print(verilog_array(f"layer{i}_biases", b))

// layer0_weights
reg signed [15:0] layer0_weights [0:127] = '{402, 728, -77, 392, -533, -701, -434, -161, -607, 373, -380, 616, -669, 445, -501, 316, 143, 307, 272, 129, 13, -324, 54, -390, -69, 256, 241, 245, -378, -153, -293, -375, 29, 426, 522, 87, 123, -236, 147, -423, 32, 292, 400, 158, -519, -242, -285, -338, -406, 558, 524, -332, 594, -517, 537, -596, 572, 599, 446, 454, -551, -576, -674, -548, 549, 645, -238, 515, -484, 56, -460, 708, -553, -367, -392, 215, -220, 813, -571, 520, 284, 4, -114, 350, -503, 368, -364, 676, -400, -482, -392, -161, 145, 602, -113, 471, -589, -439, -390, -453, 633, -542, 654, -465, 534, 606, 228, 243, 300, -615, 318, -183, -552, 824, 766, -443, 536, -702, 651, -579, 545, 674, 549, 673, -793, -823, -652, -672}};

// layer0_biases
reg signed [15:0] layer0_biases [0:15] = '{-105, -197, -19, -92, 93, 131, 47, 85, 169, -99, 46, -120, 69, -91, 159, -23}};

// layer1_weights
reg signed [15:0] layer1_weights [0:63] = '{-58, -124, 64, 251, -183, 183, 9, 253, 

# Phase-8

In [82]:
import numpy as np

# -------------------------------
# Step 1: Input EEG features
# Example: 8 features normalized 0-1 from EEG FFT / band power
# Replace with actual feature vector from your dataset
features = np.array([0.5, 0.25, 0.78, 0.39, 0.51, 0.35, 0.82, 0.33])

# -------------------------------
# Step 2: Convert features to Q8.8 fixed-point integers
features_q88 = np.round(features * 256).astype(np.int16)
print("Features Q8.8:", features_q88)

# -------------------------------
# Step 3: Define ANN weights & biases (from 8C)

# Hidden layer weights (128 values, 16 neurons x 8 inputs)
layer0_weights = np.array([
    402, 728, -77, 392, -533, -701, -434, -161,
    -607, 373, -380, 616, -669, 445, -501, 316,
    143, 307, 272, 129, 13, -324, 54, -390,
    -69, 256, 241, 245, -378, -153, -293, -375,
    29, 426, 522, 87, 123, -236, 147, -423,
    32, 292, 400, 158, -519, -242, -285, -338,
    -406, 558, 524, -332, 594, -517, 537, -596,
    572, 599, 446, 454, -551, -576, -674, -548,
    549, 645, -238, 515, -484, 56, -460, 708,
    -553, -367, -392, 215, -220, 813, -571, 520,
    284, 4, -114, 350, -503, 368, -364, 676,
    -400, -482, -392, -161, 145, 602, -113, 471,
    -589, -439, -390, -453, 633, -542, 654, -465,
    534, 606, 228, 243, 300, -615, 318, -183,
    -552, 824, 766, -443, 536, -702, 651, -579,
    545, 674, 549, 673, -793, -823, -652, -672
], dtype=np.int16)

# Hidden layer biases (16 neurons)
layer0_biases = np.array([-105, -197, -19, -92, 93, 131, 47, 85,
                          169, -99, 46, -120, 69, -91, 159, -23], dtype=np.int16)

# Output layer weights (4 neurons x 16 hidden = 64)
layer1_weights = np.array([
    -58, -124, 64, 251, -183, 183, 9, 253,
    -152, 180, 82, -150, -198, -283, 58, 241,
    175, 337, -21, -246, 128, -376, 214, -114,
    17, 266, -125, -358, 59, -511, 214, 142,
    172, 275, 12, -397, -8, 442, -174, 30,
    15, 352, 87, -330, -40, 262, -100, 249,
    273, -310, -78, -79, 23, -250, 142, 311,
    324, -118, 126, -318, 76, -403, 3, 311
], dtype=np.int16)

# Output layer biases (4 neurons)
layer1_biases = np.array([4, -6, 14, -19], dtype=np.int16)

# -------------------------------
# Step 4: Compute hidden layer outputs (16 neurons)
hidden = np.zeros(16, dtype=np.int16)

for i in range(16):
    # Weighted sum: features * weights + bias (shifted to Q8.8)
    acc = np.sum(features_q88 * layer0_weights[i*8:(i+1)*8]) + (layer0_biases[i] << 8)
    # Sigmoid approximation (same as Verilog)
    hidden[i] = (acc << 8) // (256 + abs(acc))

# -------------------------------
# Step 5: Compute output layer (4 neurons)
outputs = np.zeros(4, dtype=np.int16)

for i in range(4):
    acc = np.sum(hidden * layer1_weights[i*16:(i+1)*16]) + (layer1_biases[i] << 8)
    outputs[i] = (acc << 8) // (256 + abs(acc))

# -------------------------------
# Step 6: Print results
print("ANN outputs (Q8.8):", outputs)
print("Predicted class (argmax):", np.argmax(outputs))

Features Q8.8: [128  64 200 100 131  90 210  84]
ANN outputs (Q8.8): [-245  255 -254 -253]
Predicted class (argmax): 1


In [88]:
import numpy as np

# Replace these arrays with your trained weights/biases
layer0_weights = np.array([402, 728, -77, 392, -533, -701, -434, -161, -607, 373, -380, 616, -669, 445, -501, 316,
                           143, 307, 272, 129, 13, -324, 54, -390, -69, 256, 241, 245, -378, -153, -293, -375,
                           29, 426, 522, 87, 123, -236, 147, -423, 32, 292, 400, 158, -519, -242, -285, -338,
                           -406, 558, 524, -332, 594, -517, 537, -596, 572, 599, 446, 454, -551, -576, -674, -548,
                           549, 645, -238, 515, -484, 56, -460, 708, -553, -367, -392, 215, -220, 813, -571, 520,
                           284, 4, -114, 350, -503, 368, -364, 676, -400, -482, -392, -161, 145, 602, -113, 471,
                           -589, -439, -390, -453, 633, -542, 654, -465, 534, 606, 228, 243, 300, -615, 318, -183,
                           -552, 824, 766, -443, 536, -702, 651, -579, 545, 674, 549, 673, -793, -823, -652, -672])

layer0_biases = np.array([-105, -197, -19, -92, 93, 131, 47, 85, 169, -99, 46, -120, 69, -91, 159, -23])
layer1_weights = np.array([-58, -124, 64, 251, -183, 183, 9, 253, -152, 180, 82, -150, -198, -283, 58, 241,
                           175, 337, -21, -246, 128, -376, 214, -114, 17, 266, -125, -358, 59, -511, 214, 142,
                           172, 275, 12, -397, -8, 442, -174, 30, 15, 352, 87, -330, -40, 262, -100, 249,
                           273, -310, -78, -79, 23, -250, 142, 311, 324, -118, 126, -318, 76, -403, 3, 311])
layer1_biases = np.array([4, -6, 14, -19])

def save_mem(arr, filename):
    # Convert to 16-bit two's complement hex
    with open(filename, 'w') as f:
        for x in arr:
            if x < 0:
                x = (1 << 16) + x
            f.write(f"{x:04X}\n")

save_mem(layer0_weights, "layer0_weights.mem")
save_mem(layer0_biases, "layer0_biases.mem")
save_mem(layer1_weights, "layer1_weights.mem")
save_mem(layer1_biases, "layer1_biases.mem")

In [91]:
def export_mem_files(model):
    # model: Keras ANN
    import numpy as np

    def save_mem(arr, filename):
        with open(filename, 'w') as f:
            for x in arr.flatten():
                val = int(round(x * 256))  # convert float -> Q8.8
                if val < 0:
                    val = (1 << 16) + val
                f.write(f"{val:04X}\n")

    save_mem(model.layers[0].get_weights()[0], "layer0_weights.mem")
    save_mem(model.layers[0].get_weights()[1], "layer0_biases.mem")
    save_mem(model.layers[1].get_weights()[0], "layer1_weights.mem")
    save_mem(model.layers[1].get_weights()[1], "layer1_biases.mem")